# Projet Gestion de la Concurrence

**Kévin** **Ushaka**

**Denis** **ADANLEHOUSSI**









# Synchronizing

## Module threading

### Lock

Un verrou primitif est soit « verrouillé » soit « déverrouillé ». Il est créé dans un état déverrouillé. Il a deux méthodes, acquire() et release(). Lorsque l'état est déverrouillé, acquire() verrouille et se termine immédiatement. Lorsque l'état est verrouillé, acquire() bloque jusqu'à ce qu'un appel à release() provenant d'un autre fil d'exécution le déverrouille.

Lorsque plus d'un thread est bloqué dans acquire () en attendant que l'état passe à déverrouillé, un seul thread se poursuit lorsqu'un appel release () réinitialise l'état à déverrouillé; lequel des threads en attente continue n'est pas défini et peut varier selon les implémentations.

Pour un fonctionnement correct, il est important de libérer le verrou même si quelque chose ne va pas lors de l’accès à la ressource. On peut utiliser try-finally.

lock.acquire()
try:
    ... access shared resource
finally:
    lock.release() # release lock, no matter wha

La méthode acquire() prend un indicateur d'attente facultatif, qui peut être utilisé pour éviter le blocage si le verrou est détenu par quelqu'un d'autre.

In [ ]:
if not lock.acquire(False):
    ... failed to lock the resource
else:
    try:
        ... access shared resource
    finally:
        lock.release()

Dans l'exemple suivant, deux methodes utilisent une même varible globale g, on gère pas les sections critqiues.

Dans add_one_one() on reinitialise g à 0 puis on l'incrément de 1. Par la suite on cré une boucle de 30 avec affichage de texte juste pour faire durer l'execution de la methode . On incremente encore une fois g avant de l'afficher La resultat attendu appres cette afficheage est donc 2.

Dans add_two() on incremente juste g de 2.

Nous appelons deux threads sur ces deux methodes (add_one_one() en premier et add_two() apres).
Notons que l'on attends que les deux threads se termine ( fonction join()), pour afficher le résultat seulement lexecution totale des deux Threads.

In [1]:
#lock_tut.py
from threading import Lock, Thread
lock = Lock()
g = 0

def add_one_one():

   global g
   g=0
   g += 1
   for i in range(30):
     print('boucle d\'attente')
   g += 1
   print('Affichage de g dans add one one :',g)

def add_two():
   global g
   g += 2

threads = []
for func in [add_one_one, add_two]:
   threads.append(Thread(target=func))

threads[0].start() 
threads[1].start()

for thread in threads:
   """
   Waits for threads to complete before moving on with the main
   script.
   """
   thread.join()

print(g)

boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
Affichage de g dans add one one : 4
4


Dans l'exemple suivant, nous gèrons les sections critqiues avec le verrou threading.Lock. On obtient toujours 2 pour l'affichage dans add_one_one et 4 à la fin.

In [ ]:
#lock_tut.py
from threading import Lock, Thread
lock = Lock()
g = 0

def add_one():
   global g
   lock.acquire()
   g=0
   g += 1
   for i in range(30):
     print('boucle d\'attente')
   g += 1
   print('Affichage de g dans add one :',g)
   lock.release()

def add_two():
   global g
   lock.acquire()
   g += 2
   lock.release()

threads = []
for func in [add_one, add_two]:
   threads.append(Thread(target=func))
   threads[-1].start()

for thread in threads:
   """
   Waits for threads to complete before moving on with the main
   script.
   """
   thread.join()

print(g)

### ReLock

L'objet de verrouillage standard ne se soucie pas du thread qui détient actuellement le verrou; si le verrou est maintenu, tout thread qui tente d'acquérir le verrou se bloque, même si le même thread détient déjà le verrou. Dans l'exemple suivant, les fonctions get_first_part() et get_second_part() resteront bloquées, car la fonction externe increment() détient déjà le verrou.

In [2]:
import threading

lock = threading.Lock()

print('First try :',lock.acquire())
print('Second try:',lock.acquire(0))
print("print this if not blocked...")

First try : True
Second try: False
print this if not blocked...


La classe RLock est une version du verrouillage qui ne se bloque que si le verrou est détenu par un autre thread. Alors que les verrous simples se bloquent si le même thread tente d'acquérir le même verrou deux fois, un verrou rentrant ne se bloque que si un autre thread détient actuellement le verrou. Si le thread actuel tente d'acquérir un verrou qu'il détient déjà, l'exécution se poursuit comme d'habitude.

In [3]:
import threading

lock = threading.RLock()

print('First try :', lock.acquire())
print('Second try:', lock.acquire(blocking = True, timeout = -1))
print("print this if not blocked...")

First try : True
Second try: True
print this if not blocked...


En reprenant le même exemple mais avec un ReLock, , on réussi à réobtenir le verrou.

### Semaphore

Un sémaphore gère un compteur interne qui est décrémenté à chaque appel acquise () et incrémenté à chaque appel release (). Le compteur ne peut jamais descendre en dessous de zéro; quand acquiert () trouve qu'il est égal à zéro, il bloque, en attendant qu'un autre thread appelle release ().

In [ ]:
# -*- coding: utf-8 -*-
import threading
             
class Counter(object):
    def __init__(self, start = 0):
        self.semaphore = threading.BoundedSemaphore(value=1)
        self.value = start
    def increment(self):
        print('Waiting for a lock')
        self.semaphore.acquire()
        try:
            print('Acquired a lock')
            self.value = self.value + 1
        finally:
            print('Released a lock')
            self.semaphore.release()

def worker(c):
    c.increment()
    print('Done')


my_list=[]
counter = Counter()
for i in range(2):
    t = threading.Thread(target=worker, args=(counter,))
    my_list.append(t)
    t.start()

print('Waiting for worker threads')
main_thread = threading.currentThread()
for t in my_list:
    t.join()
print('Counter:'+ str(counter.value))

### Barrier

Une barrière est une simple primitive de synchronisation qui peut être utilisée par différents threads pour s'attendre les uns les autres. Chaque thread tente de passer une barrière en appelant la wait()méthode, qui se bloquera jusqu'à ce que tous les threads aient effectué cet appel. Dès que cela se produit, les threads sont libérés simultanément. L'extrait de code suivant illustre l'utilisation de Barrier

In [ ]:
#barrier_tut.py
from random import randrange
from threading import Barrier, Thread
from time import ctime, sleep

num = 4
# 4 threads will need to pass this barrier to get released.
b = Barrier(num)
names = ["Harsh", "Lokesh", "George", "Iqbal"]

def player():
    name = names.pop()
    sleep(randrange(2, 5))
    print("%s reached the barrier at: %s" % (name, ctime()))
    b.wait()
    
threads = []
print("Race starts now…")

for i in range(num):
    threads.append(Thread(target=player))
    threads[-1].start()
"""
Following loop enables waiting for the threads to complete before moving on with the main script.
"""
for thread in threads:
    thread.join()
print()
print("Race over!")

### Objet Event

Nous utilisons plusieurs threads pour faire tourner des opérations séparées afin de les exécuter simultanément, cependant, il y a des moments où il est important de pouvoir synchroniser les opérations de deux ou plusieurs threads. L'utilisation d' objets Event est le moyen simple de communiquer entre les threads.

Un événement gère un indicateur interne que les appelants peuvent définir () ou effacer () . D'autres threads peuvent attendre () que l'indicateur soit défini () . Notez que la méthode wait () se bloque jusqu'à ce que l'indicateur soit vrai.

In [ ]:
import threading
import time
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-9s) %(message)s',)
                    
def wait_for_event(e):
    logging.debug('wait_for_event starting')
    event_is_set = e.wait()
    logging.debug('event set: %s', event_is_set)

def wait_for_event_timeout(e, t):
    while not e.isSet():
        logging.debug('wait_for_event_timeout starting')
        event_is_set = e.wait(t)
        logging.debug('event set: %s', event_is_set)
        if event_is_set:
            logging.debug('processing event')
        else:
            logging.debug('doing other things')

if __name__ == '__main__':
    e = threading.Event()
    t1 = threading.Thread(name='blocking', 
                      target=wait_for_event,
                      args=(e,))
    t1.start()

    t2 = threading.Thread(name='non-blocking', 
                      target=wait_for_event_timeout, 
                      args=(e, 2))
    t2.start()

    logging.debug('Waiting before calling Event.set()')
    time.sleep(3)
    e.set()
    logging.debug('Event is set')

## Module multiprocessing

Le processus ont leur propre espace virtuel, ils ne pas partagent leur mémoire. C'est pourquoi le module multiprocessing gère deux types de canaux de communication entre les processus : Queue , Tubes.

In [ ]:
from multiprocessing import Process, Queue

def f(q):
    q.put([42, None, 'hello'])

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    print(q.get())    # prints "[42, None, 'hello']"
    p.join()

Pour utiliser un tube on instancie un objet Pipe. Les deux objets de connexion renvoyés par Pipe() représentent les deux bouts d'un tube. Chaque objet de connexion possède des méthodes send() et recv().

In [ ]:
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'hello'])
    conn.close()

if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(target=f, args=(child_conn,))
    p.start()
    print(parent_conn.recv())   # prints "[42, None, 'hello']"
    p.join()

Il est également possible de créer des objets partagés utilisant une mémoire partagée pouvant être héritée par les processus enfants avec multiprocessing.Value(typecode_or_type, *args, lock=True).

Les opérations telles que += qui impliquent une lecture et une écriture ne sont pas atomique. Ainsi si on souhaite par exemple réaliser une incrémentation atomique sur une valeur partagée, on ne peut pas simplement faire

In [ ]:
# -*- coding: utf-8 -*-
import multiprocessing 

# function to deposit to account 
def increment(variable):     
    variable.value = variable.value + 1

if __name__ == "__main__": 
    counter = multiprocessing.Value('i', 0)

    p1 = multiprocessing.Process(target=increment, args=(counter,)) 
    p2 = multiprocessing.Process(target=increment, args=(counter,)) 
  
    p1.start() 
    p2.start() 

    p1.join();
    p2.join();

    print("Final balance = "+str(counter.value)) 

Le paquet multiprocessing reproduit en grande partie l'API du module threading. O nretrouve les primitives de synchronisation :
multiprocessing.Lock
multiprocessing.RLock
multiprocessing.BoundedSemaphore([value])

### Lock

Un verrou non récursif : un analogue proche de threading.Lock. Une fois que le processus ou le fil d'exécution a acquis un verrou, les tentatives suivantes d'acquisition depuis n'importe quel processus ou fil d'exécution bloqueront jusqu'à ce qu'il soit libéré ; n'importe quel processus ou fil peut le libérer. Les concepts et comportements de threading.Lock qui s'appliquent aux fils d'exécution sont répliqués ici dans multiprocessing.Lock et s'appliquent aux processus et aux fils d'exécution.

In [ ]:
import multiprocessing 
  
# function to withdraw from account 
def withdraw(balance, lock):     
    for _ in range(10000): 
        lock.acquire() 
        balance.value = balance.value - 1
        lock.release() 
  
# function to deposit to account 
def deposit(balance, lock):     
    for _ in range(10000): 
        lock.acquire() 
        balance.value = balance.value + 1
        lock.release() 
  
def perform_transactions(): 
  
    # initial balance (in shared memory) 
    balance = multiprocessing.Value('i', 100) 
  
    # creating a lock object 
    lock = multiprocessing.Lock() 
  
    # creating new processes 
    p1 = multiprocessing.Process(target=withdraw, args=(balance,lock)) 
    p2 = multiprocessing.Process(target=deposit, args=(balance,lock)) 
  
    # starting processes 
    p1.start() 
    p2.start() 
  
    # wait until processes are finished 
    p1.join() 
    p2.join() 
  
    # print final balance 
    print("Final balance = "+str(balance.value)) 
  
if __name__ == "__main__": 
    for _ in range(10): 
  
        # perform same transaction process 10 times 
        perform_transactions()

### RLock

Un objet verrou récursif : un analogue proche de threading.RLock. Un verrou récursif doit être libéré par le processus ou le fil d'exécution qui l'a acquis. Quand un processus ou un fil acquiert un verrou récursif, le même processus/fil peut l'acquérir à nouveau sans bloquer ; le processus/fil doit le libérer autant de fois qu'il l'acquiert.

### Semaphore

Une seule différence existe avec son proche analogue : le premier argument de sa méthode acquire est appelé block, pour la cohérence avec Lock.acquire().